# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Zadanie domowe 2

In [1]:
import pandas as pd
import numpy as np
       

## Zadanie 1 - eksploracja przestrzeni zagnieżdżeń
Wczytajmy do przestrzeni plik zagnieżdżeń, który należy pobrać ze strony:
https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.pl.vec Są to zagnieżdzenia dla języka polskiego uzyskane systemem fastText.

Do przestrzeni wczytujemy tylko 100 tys. najczęstrzych słów, tak aby operacje przebiegały szybciej.

In [2]:
import io
def load_vectors(fname, limit = 100000):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    n = min(n,limit)
    embeddings = np.empty((n,d), dtype = np.float)
    words_idx = []
    for i, line in enumerate(fin):
        if i >= limit:
            break
        tokens = line.rstrip().split(' ')
        words_idx.append(tokens[0])
        embeddings[i] =  np.array(tokens[1:]).astype(np.float)
    return words_idx, embeddings
words_idx, embeddings = load_vectors('wiki.pl.vec')

Poniższe zadania mają na celu poekserymentowanie z przestrzenią zagnieżdżeń, ale też zrozumienie stojącymi za nich operacji. Dozwolone jest korzystanie tylko z podstawowych operatorów Python i numpy (w szczególności zakaz dotyczy: sklearn, gensim, fasttext itd.)

Jeśli potrzebujesz do dalszego przetwarzania przeprowadzenia jakichś normalizacji macierzy -- możesz wstępnie przetworzyć macierz zagnieżdzeń poniżej. Pamiętaj, że sprawdzarka będzie używała wywołań do `embeddings` (i `words_idx`) -- musisz nadpisać macierz zagnieżdzeń. To pole jest pomocnicze i nie podlega ocenie.

In [3]:
# YOUR CODE HERE

d = {word:idx for idx,word in enumerate(words_idx)}

Zaimplementuj funkcję, która obliczy podobieństwo kosinusowe pomiędzy dwoma wyrazami.

In [4]:
def cosine_sim(embed_a, embed_b):
    return np.sum(embed_a * embed_b) / (np.sqrt(np.sum(embed_a**2))   *   np.sqrt(np.sum(embed_b**2)) )

def calc_sim(word_a, word_b, words_idx, embeddings):
    embed_a = embeddings[words_idx.index(word_a)]
    embed_b = embeddings[words_idx.index(word_b)]
#     embed_a = embeddings[d[word_a]]
#     embed_b = embeddings[d[word_b]]
    return cosine_sim(embed_a, embed_b)
    

In [5]:
from nose.tools import assert_almost_equal
assert_almost_equal(calc_sim("bieber", "rihanna", words_idx, embeddings), calc_sim("rihanna", "bieber", words_idx, embeddings))

Policz podobieństwo pomiędzy wyrazem `bieber` a wyrazami:
    - `rihanna`
    - `piłsudski`
    - `kanada`
    - `polska`
    - `piosenka`

In [6]:
calc_sim("bieber", "rihanna", words_idx, embeddings)

0.524583248263655

In [7]:
# YOUR CODE HERE
[calc_sim("bieber", "rihanna", words_idx, embeddings),
calc_sim("bieber", "piłsudski", words_idx, embeddings),
calc_sim("bieber", "kanada", words_idx, embeddings),
calc_sim("bieber", "polska", words_idx, embeddings),
calc_sim("bieber", "piosenka", words_idx, embeddings)]

[0.524583248263655,
 0.19303958051463557,
 0.20042742126487928,
 0.12505934735679375,
 0.2874871368858332]

Zaimplementuj funkcję, która zwróci najbardziej podobne słowa (miara kosinusowa) do danego słowa `word`. W wyniku wypisz tylko `top` słów z najbliższymi zagnieżdżeniami, pomijając słowo `word`.

In [8]:
def find_similar_to_embedding(embed_word, words_idx, embedding_matrix, top):
    idxs = np.argsort([cosine_sim(embed_word, embedding_matrix[i]) for i in range(np.shape(embedding_matrix)[0])])[-1-top:-1][::-1]
    return np.array(words_idx)[idxs]
        

def find_similar(word, words_idx, embedding_matrix, top=10):
    embed_word = embedding_matrix[words_idx.index(word)]
    return find_similar_to_embedding(embed_word, words_idx, embedding_matrix, top)
    


In [9]:
assert len(find_similar("radość", words_idx, embeddings)) == 10

Znajdź najbardziej podobne słowa do kobieta, politechnika, mateusz, szczecin, niemcy, piłsudski

In [10]:
find_similar("kobieta", words_idx, embeddings)


array(['kobietą', 'dziewczyna', 'mężczyzna', 'kobietę', 'dziewczynka',
       'mężczyznę', 'staruszka', 'mężczyzną', 'kobiecie', 'mężczyzny'],
      dtype='<U31')

In [11]:
# YOUR CODE HERE
[find_similar("kobieta", words_idx, embeddings),
 find_similar("politechnika", words_idx, embeddings),
 find_similar("mateusz", words_idx, embeddings),
 find_similar("szczecin", words_idx, embeddings),
 find_similar("niemcy", words_idx, embeddings),
 find_similar("piłsudski", words_idx, embeddings)]

[array(['kobietą', 'dziewczyna', 'mężczyzna', 'kobietę', 'dziewczynka',
        'mężczyznę', 'staruszka', 'mężczyzną', 'kobiecie', 'mężczyzny'],
       dtype='<U31'),
 array(['politechniki', 'politechniką', 'politechnikę', 'politechniczny',
        'politechnice', 'politechnicznej', 'politechnicznego',
        'politechnicznym', 'inżynierska', 'elektrotechnika'], dtype='<U31'),
 array(['łukasz', 'bartłomiej', 'bartosz', 'kacper', 'marcin', 'mateusza',
        'tomasz', 'patryk', 'rafał', 'mateuszem'], dtype='<U31'),
 array(['szczecinek', 'szczeciński', 'szczecinem', 'gryfino', 'szczecinie',
        'stargard', 'szczecina', 'koszalin', 'szczecińska', 'świnoujście'],
       dtype='<U31'),
 array(['niemieccy', 'naziści', 'alianci', 'okupanci', 'polacy',
        'hitlerowcy', 'niemieckie', 'rosjanie', 'niemców', 'niemcom'],
       dtype='<U31'),
 array(['piłsudskim', 'piłsudskiego', 'piłsudskiemu', 'sosnkowski',
        'mościcki', 'śmigły', 'józef', 'żeligowski', 'piłsudczyków',
        '

Krótko skomentuj wyniki dla słowa `niemcy`. Które z powstałych analogii biorą się z semantycznego powiązania a które z semantycznego podobieństwa?

Semantyczne podobieństwo (jako przedstawiciele narodowości) - polacy, rosjanie.
Semantyczne podobieństwo (jako nazwa przedstawiciele dowolnej grupy ludzi) - polacy, rosjanie, naziści, alianci, okupanci, hitlerowcy.

Semantyczne powiązanie (poprzez państwo Niemcy) - niemieccy, niemieckie, niemców, niemcom.
Semantyczne powiązanie (związane z II w.ś.) - naziści, alianci, okupanci, hitlerowcy.

Zaimplementuj funkcje szukającą brakującego elementu relacji ,,`word_a` jest do `word_a2` jak `word_b` jest do...''. Funkcja powinna zwrócić 10 najbardziej pasujących słow z pominięciem słów będących jej argumentami.

In [12]:
def find_similar_pair(word_a, word_a2, word_b,  words_idx, matrix, top=10):
    embed_a = matrix[words_idx.index(word_a)]
    embed_a2 = matrix[words_idx.index(word_a2)]
    embed_b = matrix[words_idx.index(word_b)]
    
    embed = embed_a2 - embed_a + embed_b
    similar = find_similar_to_embedding(embed, words_idx, matrix, top+3)
    return [w for w in similar if w not in [word_a, word_a2, word_b]]
        

In [13]:
assert find_similar_pair( "mężczyzna", "król", "kobieta", words_idx, embeddings)[0] == "królowa"

Pieniądze są do profesora jak wiedza do...

In [14]:
# YOUR CODE HERE
find_similar_pair( "pieniądze", "profesor", "wiedza", words_idx, embeddings)

['habilitowany',
 'docent',
 'wykładowca',
 'profesorem',
 'habilitacja',
 'adiunkt',
 'rektor',
 'profesora',
 'humanistycznych',
 'naukowiec',
 'prorektor',
 'literaturoznawca']

Mateusza jest do mateusz jak łukasza do ...

In [15]:
# YOUR CODE HERE
find_similar_pair( "mateusza", "mateusz", "łukasza", words_idx, embeddings)

['łukasz',
 'bartłomiej',
 'bartosz',
 'maciej',
 'tomasz',
 'rafał',
 'patryk',
 'marcin',
 'michał',
 'przemysław',
 'łukaszewski',
 'bartczak']

Warszawa jest do "polska" jak "berlin" do ...

In [16]:
# YOUR CODE HERE
find_similar_pair( "warszawa", "polska", "berlin", words_idx, embeddings)

['niemiecka',
 'berliner',
 'wschodnioniemiecka',
 'berlińska',
 'deutschland',
 'deutsche',
 'brandenburg',
 'berlinem',
 'germany',
 'niemcy',
 'berlinie',
 'berlina']

Zurich jest do ETH jak Poznań do ...

In [17]:
find_similar_pair( "zurich", "eth", "poznań", words_idx, embeddings)

['„poznań',
 'wrocław',
 'poznania',
 'poznańskie',
 'uam',
 'poznaniu',
 'kraków',
 'gniezno',
 'poznańską',
 'wlkp',
 'wielkopolski',
 'gorzów']

Niemcy są do Merkel jak Polska do ...

In [18]:
find_similar_pair( "niemcy", "merkel", "polska", words_idx, embeddings)

['kaczyńska',
 'lewandowska',
 'kwaśniewska',
 'ekonomistka',
 'lekarka',
 'parlamentarzystka',
 'marcinkiewicz',
 'olszewska',
 'bezpartyjna',
 'dziennikarka',
 'nowacka',
 'geremek']

Na wektorach możemy wykonywać standardowe operacje algebry liniowej takie jak np. projekcja czyli rzutowanie danych na jakichś zbiór osi (więcej: notatki z algebry liniowej np. https://ocw.mit.edu/courses/mathematics/18-06sc-linear-algebra-fall-2011/least-squares-determinants-and-eigenvalues/projections-onto-subspaces/). W szczególności może to się przydać do zrzutowania słowa na przestrzeń w której pewny wybrany kierunek (wskazywany przez wektor) jest eliminowany.

Do czego może to się przydać? Jeśli uruchomisz funkcję `find_similar` dla słowa ,,mateusza'' znajdziesz m.in. ,,łukasza'' ale także ,,ewangelia'', ,,ewangelisty'' i ,,apostoła''. Chcąc pominąc kontekst religijny tego słowa możesz zrzutować jego reprezentacje na przestrzeń bez wektora ,,ewangelia'' i poszukać jego najbliższych sąsiadów (którymi będą teraz po prostu imiona męskie). Zaimplementuj taką funkcję.


In [19]:
#https://en.wikipedia.org/wiki/Orthogonal_complement
def get_orthogonal_complement_full_matrix(vectors):

    def zeroOutDimension(vectorZeroed, vectorZeroing, zeroingInd):
        vectorZeroed -= vectorZeroing * vectorZeroed[zeroingInd] / vectorZeroing[zeroingInd]

    def normalizeDimension(vectorNormalized, normalizedInd):
        vectorNormalized /= vectorNormalized[normalizedInd]

    for zeroingInd, vectorZeroing in enumerate(vectors):
        for zeroedInd, vectorZeroed in enumerate(vectors):
            if zeroingInd!=zeroedInd:
                zeroOutDimension(vectorZeroed, vectorZeroing, zeroingInd)

    for normalizedInd, vectorNormalized in enumerate(vectors):
        normalizeDimension(vectorNormalized, normalizedInd)

    matrix = np.identity(np.shape(vectors)[1])
    matrix[:, :np.shape(vectors)[0]] = np.transpose(-vectors)
    matrix[:np.shape(vectors)[0]] = vectors


    return matrix.T

def transform_vector_space(vector_space, transformation_matrix):
    return (np.linalg.inv(transformation_matrix) @ vector_space.T).T

def find_similar_with_rejection(word, remove, words_idx, matrix, top=10):
    """
    Działanie analogiczne do find_similar z dodatkowym parametrem remove, 
    który jest *listą* słów, które należy wyrzucić poprzez projekcję.
    Dla remove=[] powinno się zwracać dokładnie to samo co find_similar
    """
    if len(remove)>0:
        words_removed = np.array([embeddings[words_idx.index(wordR)] for wordR in remove])
        # setting vectors to remove as first k basis vectors of transformed space
        transformation_matrix = get_orthogonal_complement_full_matrix(words_removed)
        matrix_transformed = transform_vector_space(matrix, transformation_matrix)

        matrix_transformed = matrix_transformed[:, np.shape(words_removed)[0]:] # dropping vector dimension
    else:
        matrix_transformed = matrix
    
    return find_similar(word, words_idx, matrix_transformed, top)
    
    
print ("Standardowe poszukiwanie:", find_similar_with_rejection("mateusza",[] , words_idx, embeddings))
print ("Poszukiwanie po projekcji:", find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings))


## Zaimplementowana metoda przekształca całą przestrzeń zagnieżdżeń: vector_space = matrix
## macierzą transformacji o nowych współrzędnych n osi wybranych jako:
## k -> pierwszych osi -> kierunki, które mamy usunąć
## n-k -> pozostałych osi -> osie wyznaczone w taki sposób, aby tworzyły n-k wymiarową przestrzeń prostopadłą do k wymiarowej
## przestrzeni pierwszych osi

## Po transformacji k pierwszych kolumn w embeddingsach oznacza jak bardzo słowo jest podobne do słowa, którego embedding
## przyjęliśmy na tę oś, a embedding usuniętego słowa dla "własnej osi" ma wartość 1, dla pozostałych osi wartość 0.

## Następnie usuwamy k pierwszych kolumn/osi usuwając z przestrzeni te kierunki - posiadając n-k wymiarową
## przestrzeń prostopadłą do usuniętej przestrzeni k wymiarowej - tzn. punkty w usuniętej przestrzeni są niedostępne
## dla przestrzeni pozostałej tzn. ze wszystkich embeddingsów słów usunęliśmy całkowicie podobieństwo (wg. embeddingsów) 
## do usuniętych słów. 

## Operacja jest dość kosztowna bo wymaga przekształcenia całej macierzy embeddingsów
## oprócz tego tracimy k wymiarów embeddingsów - jest ograniczona liczba słów, którą możemy w taki sposób usunąć. 

## W związku z inną niż proponowana przez prowadzącego implementacja - słowa zwrócone zastosowaną metodą
## mogą zwracać inne słowa niż zakładane i nie przejść ukrytych testów - proszę o nie branie takich ewentualnych ukrytych testów
## pod uwagę w ocenianiu :)

Standardowe poszukiwanie: ['łukasza' 'ewangelii' 'ewangelisty' 'ewangelia' 'bartłomieja'
 'ewangeliach' 'apostoła' 'mateusz' 'tymoteusza' 'jakuba']
Poszukiwanie po projekcji: ['łukasza' 'macieja' 'bartłomieja' 'marcina' 'piotra' 'andrzeja' 'mateusz'
 'jakuba' 'tomasza' 'michała']


In [20]:
assert "ewangelii" in find_similar_with_rejection("mateusza",[] , words_idx, embeddings)
assert "ewangelii" not in find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings)
assert "ewangelisty" not in find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings)


Analogicznie słowo ,,java'' jest nie tylko nazwą języka programownia (https://pl.wikipedia.org/wiki/Java_(ujednoznacznienie)) -- jest np. nazwą geograficzną (indonezyjska wyspa koło Sumatry). Sprawdź jakie wyrazy są podobne do "java" oraz po odrzuceniu kierunku "javascript" (tj. kierunku związanego z językami programowania).

In [21]:
find_similar_with_rejection("java",["javascript"] , words_idx, embeddings)

array(['javy', 'sumatra', 'krążowniki', 'indonezja', 'c#', 'tromp', 'api',
       'niszczycielami', 'obiektowe', 'penang'], dtype='<U31')

Spróbuj poekseprymentować samemu!

In [47]:
# usuwając dodatkowo kierunek "programowanie" - pozbywamy się dodatkowo informacji o c#, api i obiektowe (pewnie programowanie)
# wygląda na to, że informacja o programowaniu była reprezentowana przez więcej aspektów niż te ukryte jedynie w "javascript"
print(find_similar_with_rejection("java",["javascript", "programowanie"] , words_idx, embeddings))


# podobny efekt uzyskujemy dla c# -> również tracimy api i obiektowe (wygląda na to, że c# jako język backendowy kojarzony
# jest bardziej z api (niż javascript?) oraz jest językiem bardziej kojarzonym z paradygmatem obiektowym)
print(find_similar_with_rejection("java",["javascript", "c#"] , words_idx, embeddings))

# dzięki usunięciu widocznych poniżej osi, najbliższe wyrazy stały się niezależne od płci 
# (pomimo podania słowa odmienionego przez płeć)
print(find_similar_with_rejection("sprzątaczka",[] , words_idx, embeddings))

print(find_similar_with_rejection("sprzątaczka",["kobieta", "mężczyzna", "ona", "on"] , words_idx, embeddings))

['sumatra' 'javy' 'penang' 'indonezja' 'niszczycielami' 'hmas'
 'krążowniki' 'jawa' 'tromp' 'emden']
['sumatra' 'niszczycielami' 'hmas' 'indonezja' 'javy' 'tromp' 'krążowniki'
 'flagowy' 'niszczyciele' 'jawa']
['kelnerka' 'sekretarka' 'policjantka' 'recepcjonistka' 'pielęgniarka'
 'asystentka' 'kucharka' 'prostytutka' 'kierowniczka' 'pracownica']
['sprzątanie' 'sprzątania' 'sekretarka' 'recepcjonistka' 'kelnerka'
 'kierowniczka' 'pielęgniarka' 'asystentka' 'nauczycielka' 'dyrektorka']


Wykonanie projekcji w przestrzeni zagnieżdżeń może być jedną z prostych technik zwalczenia tzw. gender bias (http://wordbias.umiacs.umd.edu/) w reprezentacji słów. Okazuje się, że wykonanie projekcji macierzy zagnieżdżeń na przestrzeń w której ,,brakuje kierunku he-she'' może być bardzo prostą techniką zredukowania tego typu obciążenia.

## Zadanie 2 - zagnieżdżenia dokumentów
W tym ćwiczeniu powócimy do zbioru tweetów, który analizowaliśmy w poprzednim dokumencie.

In [48]:
from helpers import DataSet
training_set = DataSet(['tweets.txt'])

Reading data set ['tweets.txt']


In [49]:
for i in training_set.tweets:
    print(i.text)
    print(i.tokens)
    print(i.clazz)
    break

dear @Microsoft the newOoffice for Mac is great and all, but no Lync update? C'mon.
['dear', '@microsoft', 'the', 'newooffice', 'for', 'mac', 'is', 'great', 'and', 'all', ',', 'but', 'no', 'lync', 'update', '?', "c'mon", '.']
negative


Tym razem do zbudowania reprezentacji będziemy używać narzędzie Universal Sentence Encoder stworzone przez Googla na bazie głębokiej sieci uśredniającej (i architektur rekurencyjnych). Poniższy kod pokazuje sposób użycia tego narzędzia. 
Kod spokojnie można wywoływać na CPU -- choć ściąganie modelu trochę może potrwać.

In [52]:
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])
print (embeddings)

tf.Tensor(
[[-0.03133017 -0.06338635 -0.01607501 ... -0.0324278  -0.04575742
   0.05370458]
 [ 0.05080862 -0.01652429  0.01573782 ...  0.0097666   0.03170121
   0.01788118]], shape=(2, 512), dtype=float32)


Wykorzystując reprezetnację USE wytrenuj wybrany klasyfikator z pakietu `sklearn` i zweryfikuj jego jakość działania.

In [53]:
# YOUR CODE HERE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# tensorflow 2.0.0 or newer
X = embed([tweet.text for tweet in training_set.tweets]).numpy()

y = [i.clazz for i in training_set.tweets]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

clf  = RandomForestClassifier(n_estimators=20, random_state=1)
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
acc = accuracy_score(y_test, pred)
print("Random Forest, accuracy: " + str(acc))

Random Forest, accuracy: 0.6009988901220865


Skomentuj wyniki i odnieś je do wyników z poprzedniego zadania domowego. Na ile użycie reprezentacji rozproszonych pozwoliło na poprawę wyników?

Użycie reprezentacji rozproszonych zwiększyło trafność klasyfikacji Random Forest (w tej samej konfiguracji) z 0.568
(dla klastrów browna), i z 0.580 (najlepsza konfiguracja hashowania) do 0.600. Jest to poprawa o odpowiednio, o około 5.5% oraz 3.5% trafności - wydaje się to dość znacząca poprawa, szczególnie, że zastosowany model nie jest wyrafinowany - możliwe, że mocniejszy model potrafiłby uzyskać jeszcze większe polepszenie wyników.

# Zadanie 3 - konstruowanie zagnieżdżeń
W tym ćwiczeniu kontynuujemy pracę z tweetami, ale pomijamy całkowicie ich klasy. Zbiór tweetów potraktujemy jako korpus do nauczenia zagnieżdżeń słów przy pomocy macierzy PMI.
- Wypełnij macierz kontekst - dokument przy użyciu symetrycznego okna o promieniu 4 (po 4 słowa w każdą stronę)
- Możesz ograniczyć słownictwo do 10K słów
- Przekształć macierz w macierz PPMI
- Stwórz zagnieżdżenia wykorzystując dekompozycję SVD do wybranej wymiarowości $d$ (ze względu na koszt obliczeniowy może to być mała wymiarowość np. $d=10$)

In [55]:
# approach inspired by 
# https://www.kaggle.com/gabrielaltay/word-vectors-from-pmi-matrix?fbclid=IwAR38v32cjpDzJZAdJBTsloAVYspANhk2Xn1CYbzX0ByhscCvR0gDKYIEUIk

from collections import Counter
import numpy as np
from scipy.sparse.linalg import svds

vocab_size = 10000
word_ctr = Counter()


def count_words(tweet, ctr):
    for word in tweet.tokens:
        ctr[word] += 1
        



for tweet in training_set.tweets:
    count_words(tweet, word_ctr)
    

token_to_ind = {}
ind_to_token = {}
# create dictionaries that convert tokens to indices and reverse
for ind, (word, count) in enumerate(word_ctr.most_common(vocab_size)):
   
    token_to_ind[word] = ind
    ind_to_token[ind] = word
    

#print(token_to_ind.keys())
    
fram_size_by_side = 4
co_occurences_ctr = Counter()


def count_co_occurences(tweet, co_occurences_ctr, token_to_ind):
    
    for ind, word in enumerate(tweet.tokens):
        
        if word not in token_to_ind:
            continue
        for context_pos in range(max(0, ind-fram_size_by_side), min(len(tweet.tokens), ind+fram_size_by_side+1)):
            # check if we hit actual word, not context
            if context_pos != ind and tweet.tokens[context_pos] in token_to_ind:
                #print((token_to_ind[word], token_to_ind[tweet.tokens[context_pos]]))
                co_occurences_ctr[(token_to_ind[word], token_to_ind[tweet.tokens[context_pos]])] +=1


for tweet in training_set.tweets:
    count_co_occurences(tweet, co_occurences_ctr, token_to_ind)

#print([(ind_to_token[w], ind_to_token[c]) for ((w, c), cnt) in co_occurences_ctr.most_common(10)])



word_context = np.zeros((vocab_size,vocab_size))
rows, cols = zip(*co_occurences_ctr.keys())
values = list(co_occurences_ctr.values())



np.add.at(word_context, (rows,cols), values)

log_word_context = np.log(word_context)


log_sum_over_words = np.log(np.sum(word_context, axis=0))
log_sum_over_contexts = np.log(np.sum(word_context, axis=1))


log_sum = np.log(np.sum(word_context))

pmi = ((log_word_context + log_sum - log_sum_over_words).transpose() - log_sum_over_contexts).transpose()

ppmi = np.where( pmi >0, pmi, 0) 

u,s, vt = svds(ppmi, 10)

#print(u.shape)
u_norm = (u.T/np.linalg.norm(u, ord=2, axis=1)).T


#print(np.sqrt(np.sum(u_norm**2, axis=1)))

print("Embedding of word: " + ind_to_token[0] + " = " + str(u_norm[0]))

new_words_idx = list(ind_to_token.values())

u_norm

c:\users\piotr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: divide by zero encountered in log


Embedding of word: the = [-0.13393392  0.17933441 -0.36174603  0.01115794  0.36489091 -0.33519542
 -0.11976622 -0.30472809 -0.59883297 -0.32804078]


array([[-0.13393392,  0.17933441, -0.36174603, ..., -0.30472809,
        -0.59883297, -0.32804078],
       [-0.35739485, -0.08010197, -0.40097614, ..., -0.00842042,
        -0.61931274, -0.31606749],
       [-0.18342168, -0.19327841, -0.23983594, ..., -0.32607908,
        -0.63501306, -0.34592036],
       ...,
       [-0.16671406, -0.30649568, -0.4776308 , ...,  0.04280096,
         0.42409233, -0.21069002],
       [-0.12806573, -0.25234931,  0.07536458, ...,  0.60488771,
         0.53215151, -0.29560054],
       [-0.20225923, -0.18782181,  0.33274312, ...,  0.59123672,
         0.43065011, -0.29270055]])

Przetestuj działanie Twoich zagnieżdżeń wykorzystując funkcję `find_similar` na wybranych słowach.

In [70]:
# zagnieżdżenia wydają się dużo słabsze (aczkolwiek też sensowne), ale z drugiej strony mamy tutaj tylko 10000 słów 
# (i zagnieżdżenia też na 10000 są zbudowane)
print(find_similar('soccer', new_words_idx, u_norm))

print(find_similar('football', new_words_idx, u_norm))


print(find_similar('harry', new_words_idx, u_norm))

['referee' 'mascot' 'attending' 'england' 'showed' 'harvey' 'football'
 'khan' 'keeps' 'expressed']
['v' '12th' 'la' 'league' 'everton' 'liverpool' 'premier' 'defender'
 'champions' '@katetscott']
['potter' 'off' 'hannibal' 'got' 'jurassic' 'had' 'watching' 'good'
 'world' 'thrones']
